In [91]:
# Package imports
import pandas as pd
import pdfplumber
import re
import ast
import numpy as np

# Variable definition
filepath = "C:/Users\lukec\OneDrive - University of Cambridge\Projects\PhD\Data\pdfs\iron ore beneficiation_IN_2017_Undefined.pdf"
keywords, split = ['Pedigree matrix','Comment'], ['Detailed Information For Exchanges','Inputs from','Emissions']

In [92]:
# Extract text from pdf
pdf_text = str(list(map(lambda x: x.extract_text(), pdfplumber.open(filepath).pages)))

text = pdf_text[list(re.finditer(split[0], pdf_text))[1].end():] # Filter out header pages
locs1, locs2 = [list(map(lambda x: x.end(), list(re.finditer(kw, text)))) for kw in keywords] # Find keyword locations
pedigrees = list(map(lambda x: ast.literal_eval(text[x+2:x+17]), locs1)) # Pedigree matrices to list

# Extract names of inputs/outputs
val_lines = list(map(lambda x: list(re.finditer(r"\\n",text[:x]))[-2].end(),locs2)) # Get values for start of section preceding keyword
val_lines = np.array(list(map(lambda x,y: list(re.finditer(r"\\n",text[:x]))[-3].end() if 'Activity' in text[y:y+10] else y, locs2, val_lines)))
val_lines += list(map(lambda x: re.search('[a-zA-z]',text[x:]).start(),val_lines)) # Find first letter (caters for new pages etc...)
num_starts = list(map(lambda x: re.search("\d+\.\d+",text[x:]).start(), val_lines)) # Find start of value for each process
num_ends = list(map(lambda x: re.search('\s',text[x:]).end(), (np.array(val_lines)+num_starts)))
components = list(map(lambda x,y: text[x:x+y],val_lines,np.array(num_starts)-1))
quantities = list(map(lambda x,y: text[x:x+y],np.array(val_lines)+num_starts,num_ends))

#info_start = list(re.finditer(split[0], text))[1].end()
input_output_switch = list(map(lambda x: list(re.finditer(x, text))[0].end(),split[1:]))
#features = list(re.finditer(keywords[0], text))

process = pd.DataFrame(np.array([components,quantities,pedigrees]).transpose(),columns=['components','quantities','pedigrees'])

In [109]:
# Extract from EcoSpold xmls

2144